In [1]:
import numpy as np
from Bio import Entrez
import gzip
import pickle

# In order to import from the python file without hassle, we add the current
# directory to the python path
import sys; sys.path.append(".")

Let's let the nice folks at NCBI know who we are.

In [2]:
Entrez.email = "sara.bizjak@student.fmf.uni-lj.si"

# Genome assembly

## Problem 1: Eulerian paths

In this exercise, we're going to look at the Eulerian path search you learned about in lectures. 

**a)**
Implement the `kmers` function in `helper_functions.py`. Look at the function docstrings for expected input paramters and return types. Your kmers function should be able to produce kmers of arbitrary size e.g. if we want 7-mers for the sequence *mesenchyme*, the function should return a list containing the following sequences: *mesench*, *esenchy*, *senchym*, *enchyme*. Your function should support arbitrary strides, e.g. in the previous example we used stride 1, while if we used the same string with stride 2, we would obtain the strings *mesench* and *senchym*.
**[5 points]**

**b)**
Implement the `assemble_genome` function in `helper_functions.py`. Look at the function docstrings for expected input paramters and return types.  Your genome assembly function should work in the following way:
1. For an input list of sequences, e.g. kmers, construct a DeBruijn graph as seen in the lectures.
2. Find all possible Euerlian paths through the graph, i.e. all possible paths which visit each edge exactly once. Your paths should all start from a source node with in-degree zero. In case no such node exists, you may use the first sequence in the list of input sequences as your starting point. 
3. Decode your obtained paths into sequences, and return a list (or set) of unique genome assemblies as strings.
**[10 points]**

**c)**
Modify your `assemble_genome` function in `helper_functions.py` to accept a list of input sequences in any order and still produce a correct reconstruction. An easy way to check this is to construct a list of kmers, and shuffle it. Your function should return the same results as in part b), where the kmers are ordered.
**[5 points]**

In [3]:
from helper_functions import kmers, assemble_genome

## Problem 2: Minimum fragment length

We will follow the idealized example from lectures where we have perfect coverage and each kmer is assumed to be sequenced exactly once. However, as we have seen in the lectures, some assemblies do not always produce unambiguous results. Sometimes more than one reconstruction is valid! In this exercise, we want to determine the minimum fragment length necessary for unambiguous reconstruction.

**TASK:** Simulate the sequencing process by running your `kmers` function on the SARS-CoV-2 spike protein nucleotide sequence. Next, find all the valid assemblies of your obtained kmers. You'll notice that there are no ambiguities for longer kmers e.g. 50-mers, but we run into them when generating shorter "reads" (kmers). What is the minimum fragment length where we can unambiguously reconstruct the initial spike protein nucleotide sequence? In other words, what is smallest k for k-mers where we obtain exactly one valid assembly from our `assemble_genome` function? Save your answer into the `minimum_fragment_length` variable. Write your observations into the `minimum_fragment_length_comments` variable.

**[10 points]**

In [4]:
# Spike protein : "NC_045512"

with open('data/{}.pkl.gz'.format("NC_045512"), 'rb') as f:
    rec = pickle.load(f)   
    spike = rec.seq[21562:25384]
    
    for r in rec.features:
        if r.type == "CDS":
                # find the region that codes for the "S" gene
                if "S" in r.qualifiers.get("gene", []):
                    location = r.location
                    print(location)             

[21562:25384](+)


In [16]:
for i in range(50, 0, -1):
    k_kmers = kmers(spike, i, 1)
    assemble = assemble_genome(k_kmers, i, 1)
    if len(assemble) > 1:
        index = i
        break

In [18]:
# First index with len > 1 -- we are looking for last, so our index is : index + 1
index + 1

13

In [5]:
k_kmers1 = kmers(spike, 13, 1)

In [6]:
assemble1 = assemble_genome(k_kmers1, 13, 1)

In [7]:
len(assemble1)

1

In [8]:
k_kmers2 = kmers(spike, 12, 1)

In [9]:
assemble2 = assemble_genome(k_kmers2, 12, 1)

In [11]:
len(assemble2)

2

In [40]:
minimum_fragment_length = 13
minimum_fragment_length_comments = """
Its more likely for longer kmers to obtain exactly one valid assembly (to have only one Eulerian path), since for shorter ones its more likely to get the matching we are looking for --
so there is bigger probability that graph have more connections and consequently -- more Eulerian paths.
"""

## Problem 3: Unrealistic assumptions

In this homework, we are assuming perfect coverage, which means we assume every kmer will be sequenced from the genome exactly once. We simulate this by running our kmers function on a known genome sequence.

In the real world, these assumptions would hold no water. Many things have to be taken into consideration. For example, biologists first prepare the tissue to be sequenced. Imagine someone is preparing to sequence some mouse tissue, but they forget to wear gloves and contaminate the sample with some of their own cells, and bacteria from their hands. Then the mouse sample may contain human and bacterial DNA instead of just mouse DNA. Next, the sequencing process splits the DNA molecules into short fragments at random, so we might have the same region sequenced twice, while some regions will be completely skipped. Finally, the sequencing process itself works something like this: for every nucleotide in a sequence there exists some colored reagent which binds to it. So every base, ACTG, is assigned its own color. Each nucleotide then goes under a large microscopic camera, which literally takes pictures of every nucleotide, one by one. Then, this sequence of colors is converted into a nucleotide sequence. However, sometimes reagents don't bind to the nucleotides perfectly, so this can cause ambiguities or errors in the reads.

**Question:** Write down at least **three** things we might encounter in the real world for which we have not accounted for by using kmers (you may use some issues I listed above, but think of at least one other thing we haven't mentioned yet). For each of the issues, think about how you would approach each problem. You don't need to write down a full guide, just a general overview of how we might solve each problem. Write your answer into the `unrealistic_assumptions` variable.
**[5 points]**

In [42]:
# Souce : https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-3

In [ ]:
unrealistic_assumptions = """
Assumption 1: DNA is double-stranded, so while assembling strand of a genome we will not know what to use -- a read or its reverse complement.
We could do both and than choose the right one as described in "assembly_with_alignment".


Assumption 2: The reads sequencing that are generated with sequencing machines often cointain errors. 
Those errors often complicate genome assembly because it's hard to identify all overlapped reads.
The solution to this problem can be similar that the one described in the "Assumption 3". We coul allow more connections (with some error th).

Assumption 3: Some regions of the genome may not be covered by any reads. It makes impossible to reconstruct the entire genome. It may cause some "gaps" in coverage 
and our presented graph could have some missing edges/connections (may not find the right Eulerian path).
We could solve this "gap" problem by allowing more connections (not just the perfect and right ones), for example 
by setting some error threshold that would determine how many "letters" could differ to still make the connection between edges in graph.

"""

**Question:** In all of the cases we have talked about above, we pretend we are going into the genome assembly process blind. We have some DNA fragments about which we know nothing, so we have to perform assembly from the ground up. However, often times, we know something about our organisms. For example, for SARS-CoV-2, it was suspected that it was related to SARS and MERS. Think about how we could use existing genomes from related species to facilitate genome assembly. Think back to homework 2, where we looked at sequence alignment tools. Could we use some of that machinery to help with assembling the SARS-CoV-2 genome? Write down how you would approach genome assembly for SARS-CoV-2, known that it's closely related to SARS, or even the bat RaTG13. Write your answer into the `assembly_with_alignment` variable.
**[5 points]**

In [ ]:
assembly_with_alignment = """
In real life it is expected to get multiple solutions -- multiple assembling ("multiple output strings from our function assemble_genome"). 
We would like to eliminate the less accurate assemblings and keep only the "good" one.
To check which the best, we could do the sequence alignment (use the function from HW2) and choose the solutiona by comparing the scores from sequence alignment (choose the one with max score)
in comparison to RaTG13 (or SARS) -- because they are "closest" to SASR-CoV-2 virus.

I am also thinking that genome could be assembled in multiple parts with the help of function local alignment on those smaller parts compared to RaTG13 (or SARS).
After assembling and alignment of smaller parts, we would "glue" them together.
"""